In [165]:
from PIL import Image, ImageDraw
import numpy as np
import math
import PIL.ImageFont as ImageFont
np.set_printoptions(precision = 2)

In [2]:
im = Image.open("graph.png").convert("RGB")
#get size
wh = im.size
print("Size: ",wh)
#im.show()



Size:  (197, 255)


In [3]:
draw = ImageDraw.Draw(im)
draw.point((100, 100), 'red')
draw.ellipse((50,50,60,60), fill = "red")
im.show()

In [4]:
top = (150, 50)
left = (100, 125)
right = (200, 125)

line_color = (0, 0, 200)

d.line([top, left, right, top], fill=line_color, width=2)

im.save("drawn_grid.png")

NameError: name 'd' is not defined

In [288]:
def getDist(pt1, pt2):
    xdist = pt1[0]-pt2[0]
    ydist = pt1[1] - pt2[1]
    xdist = xdist*xdist
    ydist = ydist*ydist
    distance = math.sqrt(xdist+ydist)
    return distance

def getDistFromLine(line=[0,0,0], pt = [0,0]):
    numerator = abs(line[0]*pt[0]+line[1]*pt[1]+line[2])
    denominator = math.sqrt(line[0]**2+line[1]**2)
    #print(numerator, denominator)
    if(denominator==0):
        denominator = 1
    return (numerator*1.0)/denominator

def getSlope(pt1, pt2):
    ydiff = pt2[1]-pt1[1]
    xdiff = pt2[0]-pt1[0]
    return (ydiff*1.0)/xdiff


def getLineFromTwoPoints(pt1, pt2):
    m = getSlope(pt1, pt2)
    a = m
    b = -1
    c = -m*pt1[0]+pt1[1]
    print("Line: {0:0.1f}x+{1:0.1f}y+{2:0.1f}c=0".format(a,b,c))
    return [a, b, c]

def findYCoordinate(line, x):
    y = (-1.0*line[2]-x*line[0])/line[1]
    return y

def findPtAtDistOnLine(line, pt, targetDistance):
    eps = 2
    for x in range(0, 4000):
        y = findYCoordinate(line, x)
        dist = getDist(pt, [x,y])
        if(abs(targetDistance-dist)<eps):
            return [x,y]
    return [0,0]

def getAngleBetweenTwoLines(line1,line2):
    m1 = (-1.0*line1[0]/line1[1])
    m2 = (-1.0*line2[0]/line2[1])
    tantheta = abs((m2-m1)/(1+m1*m2))
    theta = math.degrees(math.atan(tantheta))
    #print(m1, m2, tantheta, theta)
    return theta

In [296]:
#Patient Image
ptlarge = Image.open("pt1large.JPG").convert("RGB")
wh = ptlarge.size
print("Size: ", wh)
ptlarge = ptlarge.rotate(90)
ptsmall = ptlarge.resize((1210,907),Image.ANTIALIAS)
ptsmall.save("pt1small.jpg")

Size:  (4032, 3024)


In [290]:
ptsmall = Image.open("pt1small.jpg").convert("RGB")
ptsmall.show()

In [291]:
#Draw points on the image
r = 10
x1,y1 = 1670,1270
x2,y2 = 1880,1350
x3,y3 = 1880,1430
x4,y4 = 1750,1440
x5,y5 = 1690,1360
x6,y6 = 1900,1360

pt1 = [1660, 1260, 1680, 1280]
pt2 = [1870, 1340, 1890, 1360]
pt3 = [1870,1420,1890,1440]
line_color = "yellow"
font_size = 60
font = ImageFont.truetype("Arial.ttf", font_size)

In [292]:
#draw on image
ptlarge = Image.open("pt1large.JPG").convert("RGB")
ptdraw = ImageDraw.Draw(ptlarge)
ptdraw.ellipse([x1-r, y1-r, x1+r, y1+r], fill = "red")
ptdraw.ellipse([x2-r, y2-r, x2+r, y2+r], fill = "red")
ptdraw.ellipse([x3-r, y3-r, x3+r, y3+r], fill = "red")
ptdraw.ellipse([x4-r, y4-r, x4+r, y4+r], fill = "red")
ptdraw.ellipse([x5-r, y5-r, x5+r, y5+r], fill = "red")
ptdraw.ellipse([x6-r, y6-r, x6+r, y6+r], fill = "red")

ptlarge.show()

In [293]:
#Coordinate Geometry
philtralLen = getDist([x1,y1],[x2,y2])
print("Length of normal philtral column: {0:0.2f}".format(philtralLen))
ptdraw.line([x1,y1,x2,y2], fill=line_color, width = 4)
midLine = getLineFromTwoPoints([x5,y5],[x6,y6])
ptdraw.line([x5,y5,x6,y6], fill=line_color, width=4)
#ptlarge.show()
cleftLen = getDist([x3,y3],[x4,y4])
deficitLen = philtralLen - cleftLen
print("Length deficit: {0:0.2f}".format(deficitLen))
distFromMidLine = getDistFromLine(midLine, [x3,y3])
print("Perpendicular distance from midline: {0:0.2f}".format(distFromMidLine))


Length of normal philtral column: 224.72
Line: 0.0x+-1.0y+1360.0c=0
Length deficit: 94.34
Perpendicular distance from midline: 70.00


In [294]:
#Find target point on line
targetPtOnLine = findPtAtDistOnLine(midLine, [x3,y3], deficitLen)
backCutLine = getLineFromTwoPoints([x3,y3], targetPtOnLine)
ptdraw.line([x3,y3,targetPtOnLine[0], targetPtOnLine[1]], fill=line_color, width=8)


Line: 1.1x+-1.0y+-563.9c=0


In [295]:
#Find angle betweeen C flap line and back cut line
CFlapLine = getLineFromTwoPoints([x4,y4],[x3,y3])
BackCutAngle = getAngleBetweenTwoLines(CFlapLine, backCutLine)
ptdraw.line([x3,y3,x4,y4], fill=line_color, width=8)
ptdraw.arc([1820, 1380, 1920, 1480], start = 180, end = 180+BackCutAngle, fill="white", width = 8)
print("BackCutAngle: {0:0.1f} degrees".format(BackCutAngle))
ptdraw.text((1600, 1100), "Back cut angle {0:0.2f} degrees".format(BackCutAngle), font = font)
ptdraw.text((x1,y1), "1", font = font)
ptdraw.text((x2-30,y2-60), "2", font = font)
ptdraw.text((x3,y3), "3", font = font)
ptdraw.text((x4,y4), "4", font = font)
ptdraw.text((x5,y5), "5", font = font)
ptdraw.text((x6,y6), "6", font = font)
ptdraw.text((targetPtOnLine[0]-10,targetPtOnLine[1]-60),"7",font=font)

ptlarge.show()

Line: -0.1x+-1.0y+1574.6c=0
BackCutAngle: 51.1 degrees
